In [1]:
import cv2 as cv
import numpy as np
import time

# Invisibility Cloak using HSV Color Space (For Color Detection) and Segmentation with OpenCV

In [3]:
video = cv.VideoCapture('x6.mp4') #Capture/Import Video

fourcc = cv.VideoWriter_fourcc(*'XVID') #FourCC video format for output
out = cv.VideoWriter('output.avi', fourcc, 30, (1920, 1080)) #arg1=Output_file_name, arg2=file_format, arg3=FPS, arg4=width_&_Height (Note:arg3 & arg4 must be same as input file) 

time.sleep(3) #give some time for the camera to warm-up

background = 0

for i in range(30): #getting 30th frame for the background
    check, background = video.read()

#Optional:
#Creating Trackbars to get correct HSV values (if HSV values are known then leave the code related to trackbars)
def any_function(x): #Temporary function to be used in cv.createTrackbar()
    None

cv.namedWindow('Trackbars') #sapereate window for trackbars
cv.createTrackbar('LH', 'Trackbars', 75, 180, any_function) #Lower Hue
cv.createTrackbar('LS', 'Trackbars', 138, 255, any_function)
cv.createTrackbar('LV', 'Trackbars', 5, 255, any_function)

cv.createTrackbar('UH', 'Trackbars', 105, 180, any_function)
cv.createTrackbar('US', 'Trackbars', 255, 255, any_function)
cv.createTrackbar('UV', 'Trackbars', 95, 255, any_function)


while 1: #if no frame then break
    check, frame = video.read()
    if not check:
        break

    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV) #converting current frame to HSV

    l_h = cv.getTrackbarPos('LH', 'Trackbars') #getting value/position from trackbar
    l_s = cv.getTrackbarPos('LS', 'Trackbars')
    l_v = cv.getTrackbarPos('LV', 'Trackbars')
    
    u_h = cv.getTrackbarPos('UH', 'Trackbars')
    u_s = cv.getTrackbarPos('US', 'Trackbars')
    u_v = cv.getTrackbarPos('UV', 'Trackbars')
    
    lower_red = np.array([l_h, l_s, l_v]) #lower boundary of HSV color (to be detected)
    upper_red = np.array([u_h, u_s, u_v])
    
#if lower & Upper HSV values are known then use blow code:
#     lower_red = np.array([60, 150, 0]) #lower boundary of HSV color (to be detected)
#     upper_red = np.array([140, 255, 255])
    
    mask = cv.inRange(hsv, lower_red, upper_red)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((6 ,6), np.uint8))
    mask = cv.morphologyEx(mask, cv.MORPH_DILATE, np.ones((12, 12), np.uint8))
    
    invetred_mask = cv.bitwise_not(mask) # 1s become 0 & 0s become 1

    masked_frame = cv.bitwise_and(frame, frame, mask=invetred_mask)

    masked_background = cv.bitwise_and(background, background, mask=mask)

    final_output = cv.addWeighted(masked_frame, 1, masked_background, 1, 0)
    
    img = cv.resize(final_output, (960, 540)) # Resize image for displaying (final_output will not be affected)
    cv.imshow("output", img)
#     cv.imshow('mask', cv.resize(mask, (960, 540))) 
#     cv.imshow('inverted_mask', invetred_mask)

    out.write(final_output) #write final frame to output file (Created earlier)

    if(cv.waitKey(1)== ord(' ')): #space_bar will break the loop
        break
    
video.release()
out.release()
cv.destroyAllWindows()

# All the blow code blocks are to get correct HSV values. (to be use in above code)

## Blow code block is to get specific frame & save as .jpg

In [88]:
#can get background frame & object/cloak frame as .jpg
cap = cv.VideoCapture('x6.mp4')
cap.set(1, 300) #300 is the frame #, to get frame # multiply FPS by time (in seconds)
_, frame = cap.read()
cap.release()
cv.imwrite('frame.jpg', frame) #writing the frame into an image

True

## Getting correct values using trackbars

In [4]:
bg = cv.imread('bg.jpg') #in bg.jpg you can add a colored layer with 50% Opacity (Using PhotoShop) to see the difference. 
fg = cv.imread('frame.jpg')

def any_function(x):
    None
    
cv.namedWindow('Trackbars')
cv.createTrackbar('LH', 'Trackbars', 75, 180, any_function)
cv.createTrackbar('LS', 'Trackbars', 138, 255, any_function)
cv.createTrackbar('LV', 'Trackbars', 5, 255, any_function)

cv.createTrackbar('UH', 'Trackbars', 105, 180, any_function)
cv.createTrackbar('US', 'Trackbars', 255, 255, any_function)
cv.createTrackbar('UV', 'Trackbars', 95, 255, any_function)

while True:    
    hsv = cv.cvtColor(fg, cv.COLOR_BGR2HSV)

    l_h = cv.getTrackbarPos('LH', 'Trackbars') #getting value/position from trackbar
    l_s = cv.getTrackbarPos('LS', 'Trackbars')
    l_v = cv.getTrackbarPos('LV', 'Trackbars')
    
    u_h = cv.getTrackbarPos('UH', 'Trackbars')
    u_s = cv.getTrackbarPos('US', 'Trackbars')
    u_v = cv.getTrackbarPos('UV', 'Trackbars')
    
    lower_red = np.array([l_h, l_s, l_v]) #lower boundary of HSV color (to be detected)
    upper_red = np.array([u_h, u_s, u_v])
    
    mask = cv.inRange(hsv, lower_red, upper_red)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, np.ones((6 ,6), np.uint8))
    mask = cv.morphologyEx(mask, cv.MORPH_DILATE, np.ones((12, 12), np.uint8))

    inverted_mask = cv.bitwise_not(mask) # 1s will be 0 & 0s will be 1

    masked_frame = cv.bitwise_and(fg, fg, mask=inverted_mask)

    masked_background = cv.bitwise_and(bg, bg, mask=mask)

    final_output = cv.addWeighted(masked_frame, 1, masked_background, 1, 0)
    
    img = cv.resize(final_output, (960, 540)) # Resize image only for display
    cv.imshow("output", img)
#     cv.imshow("final", final_output)
#     cv.imshow('HSV', hsv)

    if(cv.waitKey(1)== ord(' ')):
        break
    
cv.destroyAllWindows()